In [1]:
#import libraries
from scipy.stats import gaussian_kde
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import mutual_info_score

#define hyperparameters
num_clients = 10

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255

def ar_list(w1_np_array):
  w1_shard1 = np.reshape(w1_np_array[0:288], (3,3,1,32))
  w1_shard2 = np.reshape(w1_np_array[288:320], (32,))
  w1_shard3 = np.reshape(w1_np_array[320:18752], (3,3,32,64))
  w1_shard4 = np.reshape(w1_np_array[18752:18816], (64,))
  w1_shard5 = np.reshape(w1_np_array[18816:34816], (1600,10))
  w1_shard6 = np.reshape(w1_np_array[34816:34826], (10,))
  w1_list = [w1_shard1,w1_shard2,w1_shard3,w1_shard4,w1_shard5,w1_shard6]
  return w1_list

# Split data into shards for each client
data_per_client = len(x_train) // num_clients
x_train_shards = np.split(x_train[:num_clients * data_per_client], num_clients)
y_train_shards = np.split(y_train[:num_clients * data_per_client], num_clients)

def ar_weights(w1):
  w1_reshaped = [np.reshape(w, (-1, 1)) for w in w1]
  # Concatenate the reshaped arrays
  w1_array = np.concatenate(w1_reshaped)
  #  Convert the w1_array to a NumPy array
  w1_np_array = np.array(w1_array)
  return w1_np_array

#algorithm 1, mifl update
# Define some hyperparameters
E = 1 # Number of local epochs
B = 32 # Batch size
h = 0.01 # Learning rate
s = 0.5 # Threshold for cv
lamb = 0#lambda
omega = 0
def create_model():
  model = keras.Sequential(
      [
          keras.Input(shape=(28, 28, 1)),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(10, activation="softmax"),
      ]
  )
  return model

# Initialize global model
global_model = create_model()
global_model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(h), metrics=["accuracy"])
global_model.set_weights(ar_list(np.random.rand(34826, 1)))

def mutual_information_score(y_true, y_pred):
  # Compute the mutual information between the true labels and the predictions
  mi = mutual_info_score(y_true, y_pred)
  # Return the mutual information
  return mi

import numpy as np

'''def estimate_pdf(weights):
    # This function estimates the probability density function using Gaussian Kernel Density Estimation
    kde = gaussian_kde(weights)
    return kde

def kl_divergence(p, q):
    # This function calculates the Kullback-Leibler divergence between two distributions
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def mutual_information_score(weights1, weights2):
    # Estimate the PDFs for both sets of weights
    pdf1 = estimate_pdf(weights1)
    pdf2 = estimate_pdf(weights2)

    # Evaluate the PDFs on a grid of points (for simplicity, we assume 1D weights)
    grid_points = np.linspace(min(min(weights1), min(weights2)), max(max(weights1), max(weights2)), num=100)
    p1 = pdf1(grid_points)
    p2 = pdf2(grid_points)

    # Calculate the joint distribution as the outer product of the two PDFs
    joint_pdf = np.outer(p1, p2)

    # Calculate the product of the marginals
    product_marginals = p1[:, None] * p2[None, :]

    # Calculate the mutual information
    mi = kl_divergence(joint_pdf.ravel(), product_marginals.ravel())

    return mi'''


#list for storing current local models
local_models = []

def cv_fn(lg, lk):
  # Use the formula from the paper to calculate cv
  '''lg_mean = tf.reduce_mean(lg, axis=0)
  lk_mean = tf.reduce_mean(lk, axis=0)
  lg_var = tf.math.reduce_variance(lg, axis=0)
  lk_var = tf.math.reduce_variance(lk, axis=0)
  lg_lk_cov = tf.reduce_mean((lg - lg_mean) * (lk - lk_mean), axis=0)
  rho = (lg_var + lk_var - 2 * lg_lk_cov) / ((lg_var * lk_var)**2)'''
  correlation_matrix = np.corrcoef(lg, lk)
  rho = correlation_matrix[0, 1]
  return rho

def mi_fn(lg, lk):
  # Use the formula from the paper to calculate cv
  '''lg_mean = tf.reduce_mean(lg, axis=0)
  lk_mean = tf.reduce_mean(lk, axis=0)
  lg_var = tf.math.reduce_variance(lg, axis=0)
  lk_var = tf.math.reduce_variance(lk, axis=0)
  lg_lk_cov = tf.reduce_mean((lg - lg_mean) * (lk - lk_mean), axis=0)
  rho = (lg_var + lk_var - 2 * lg_lk_cov) / ((lg_var * lk_var)**2)'''
  correlation_matrix = np.corrcoef(lg, lk)
  rho = correlation_matrix[0, 1]
  mi = -0.5 * tf.math.log(1-rho**2)
  return mi



def mifl_update(k,t,wt):
  if(t == 0):
    # Create a local model for the client
    Fg = create_model()
    Fg.set_weights(global_model.get_weights())
    Fg_pred_array = np.argmax(Fg.predict(x_train_shards[k]), axis=1)
    lg = Fg_pred_array - y_train_shards[k]

    Fk = create_model()
    Fk.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    # Copy the global model weights to the local model
    #Fg.set_weights(global_model.get_weights())
    Fk.fit(x_train_shards[k], y_train_shards[k], batch_size=B, epochs=E, verbose=0)
    # Add the local model to the dict
    local_models.append(Fk.get_weights())
    Fk_pred_array = np.argmax(Fk.predict(x_train_shards[k]), axis=1)
    lk = Fk_pred_array - y_train_shards[k]

    mi = mi_fn(lg,lk)
    MI.append(mi)

  else:
    Fk = create_model()
    Fk.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    Fk.set_weights(local_models[k])
    Fg = create_model()
    Fg.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    weights = global_model.get_weights()
    Fg.set_weights(weights)
    #currently not taking batches into account
    for i in range (E):
      '''range is 6000 because of number of samples in each x_train_shard,y_train_shard, change it accordingly when batches have been taken into account
      for j in range(0,6000):
          lg = Fg.predict(x_train_shards[j]) - y_train_shards[j]
          lk = Fk.predict(y_train_shards[j]) - y_train_shards[j]
      '''
      Fg_pred_array = np.argmax(Fg.predict(x_train_shards[k]), axis=1)
      lg = Fg_pred_array - y_train_shards[k]

      Fk_pred_array = np.argmax(Fk.predict(x_train_shards[k]), axis=1)
      lk = Fk_pred_array - y_train_shards[k]

      cv = cv_fn(lg,lk)
      #cv = mutual_information_score(lg,lk)
      if np.array_equal(lg, lk):
        lamb = 1/2
      elif cv <= 1/2:
        lamb = cv
      else:
        lamb = 1 - cv
      omega = (1-lamb)*lg + lamb*lk

      local_weight = ar_weights(Fk.get_weights())
      # update local models
      local_weight = local_weight - h*omega
      local_weight = ar_list(local_weight)

    Fk.set_weights(local_weight)
    local_models[k] = Fk.get_weights()
    Fg_pred_array = np.argmax(Fg.predict(x_train_shards[k]), axis=1)
    lg = Fg_pred_array - y_train_shards[k]
    Fk_pred_array = np.argmax(Fk.predict(x_train_shards[k]), axis=1)
    lk = Fk_pred_array - y_train_shards[k]
    cv = cv_fn(lg,lk) # cv for updated model
    #mi = mi_fn(lg,lk)
    mi = -0.5 * tf.math.log(1-cv**2)
    MI[k] = mi
    return local_weight,mi


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
np.shape(Fg.predict(x_train_shards[k]))

NameError: name 'Fg' is not defined

In [2]:
import random
# Define the MIFL-Aggregation function
t_acc = 95#targeted accuracy
wt = [global_model.get_weights()]*num_clients
MI = []
pairs = []
ktoplist = []
Ktop = len(wt)//3#arbitrary integer for now
C = 0.1
accuracy = 0
n = int(len(wt) * C)
def mifl_aggregation(wt, MI, pairs):
  ###initialize w0
  # Run on FL server
  # wt is a list of local model weights from clients
  # MI is a list of mutual information values from clients
  t = 1 #round
  global accuracy
  while accuracy < t_acc:
    '''selected_elements = random.choices(wt, k=n)
    for k in range(len(selected_elements)):
      j = selected_elements[k]
      wt[j],MI[k] = mifl_update(k,t,wt[k])'''

    selected_nodes = random.choices([x for x in range(num_clients)], k=n)
    for k in selected_nodes:
      wt[k],MI[k] = mifl_update(k,t,wt[k])

    for k in selected_nodes:
      pairs.append([MI[k],wt[k],k])
    sorted_pairs = sorted(pairs, key=lambda x: x[0])
    # when running the algorithm for the mnist dataset, we have divided the data such that there is an
    # equal number of samples for each client, update code if that is not the case.
    for k in range(Ktop):
      ktoplist.append(ar_weights(sorted_pairs[k][1]))

    averaged_weights = np.average(ktoplist, axis=0)
    averaged_weights = ar_list(averaged_weights)
    #for k in range(Ktop,len(sorted_pairs)):
     # sorted_pairs[k][1] = averaged_weights
    #pairs = sorted(sorted_pairs, key = lambda x: x[2])
    global_model.set_weights(averaged_weights)
    accuracy = global_model.evaluate(x_test, y_test, verbose=0)[1]
    t = t + 1

for i in range (num_clients):
  mifl_update(i,0,wt)
  wt[i] = local_models[i]



'''for k in range (len(local_models)):
  wt[i] = local_models[k].get_weights()'''

#problems currently identified in second algo-
#initialize w0 not done, what do i initialize them with?
#Ktop arbitrarily taken, step 9 -TEMP FIX
#assumed each client has equal number of samples -TEMP FIX
#once sorted, take them back to old position -SOLVED
#how is accuracy being updated, not mentioned in the algo -SOLVED
#prevent overwriting
#problems in first algorithm
#didnt take batches into account
#have to see how to integrate the first and the second



188/188 [==============================] - 1s 7ms/step


'for k in range (len(local_models)):\n  wt[i] = local_models[k].get_weights()'

In [ ]:
mifl_aggregation(wt, MI, pairs)

188/188 [==============================] - 2s 8ms/step


AttributeError: 'list' object has no attribute 'predict'

In [ ]:
#import libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mutual_info_score

#define hyperparameters
num_clients = 10

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255

# Split data into shards for each client
data_per_client = len(x_train) // num_clients
x_train_shards = np.split(x_train[:num_clients * data_per_client], num_clients)
y_train_shards = np.split(y_train[:num_clients * data_per_client], num_clients)

#algorithm 1, mifl update
# Define some hyperparameters
E = 10 # Number of local epochs
B = 32 # Batch size
h = 0.01 # Learning rate
s = 0.5 # Threshold for cv
lamb = 0#lambda
omega = 0
def create_model():
  model = keras.Sequential(
      [
          keras.Input(shape=(28, 28, 1)),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(10, activation="softmax"),
      ]
  )
  return model

# Initialize global model
global_model = create_model()
global_model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(h), metrics=["accuracy"])

def mutual_information_score(y_true, y_pred):
  # Compute the mutual information between the true labels and the predictions
  mi = mutual_info_score(y_true, y_pred)
  # Return the mutual information
  return mi
#list for storing current local models
local_models = [0]*num_clients

def cv_fn(lg, lk):
  # Use the formula from the paper to calculate cv
  lg_mean = tf.reduce_mean(lg, axis=0)
  lk_mean = tf.reduce_mean(lk, axis=0)
  lg_var = tf.math.reduce_variance(lg, axis=0)
  lk_var = tf.math.reduce_variance(lk, axis=0)
  lg_lk_cov = tf.reduce_mean((lg - lg_mean) * (lk - lk_mean), axis=0)
  rho = (lg_var + lk_var - 2 * lg_lk_cov) / ((lg_var * lk_var)**2)
  cv = -0.5 * tf.math.log(1-rho**2)
  return cv

def mifl_update(k,t,wt):
  if(t == 0):
    # Create a local model for the client
    Fg = create_model()
    Fg.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    # Copy the global model weights to the local model
    Fg.set_weights(global_model.get_weights())
    Fg.fit(x_train_shards[k], y_train_shards[k], batch_size=B, epochs=E, verbose=0)
    # Add the local model to the dict
    local_models[k] = Fg
  else:
    Fk = local_models[k]
    Fg = create_model()
    Fg.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    weights = global_model.get_weights()
    Fg.set_weights(weights)
    #currently not taking batches into account
    for i in range (E):
      '''range is 6000 because of number of samples in each x_train_shard,y_train_shard, change it accordingly when batches have been taken into account
      for j in range(0,6000):
          lg = Fg.predict(x_train_shards[j]) - y_train_shards[j]
          lk = Fk.predict(y_train_shards[j]) - y_train_shards[j]
      '''
      lg = Fg.predict(x_train_shards) - y_train_shards
      lk = Fk.predict(x_train_shards) - y_train_shards
      cv = cv_fn(lg,lk)
      #cv = mutual_information_score(lg,lk)
      if lg == lk:
        lamb = 1/2
      elif cv <= 1/2:
        lamb = cv
      else:
        lamb = 1 - cv
      omega = (1-lamb)*lg + lamb*lk
      weights = weights - h*omega
    #prediction = model.predict(input_sample)
    Fg.set_weights(weights)
    mi = mutual_information_score(Fg.get_weights(),Fk.get_weights())
    return weights,mi

import random
# Define the MIFL-Aggregation function
t_acc = 95 #targeted accuracy
wt = [0]*num_clients
MI = []
pairs = []
ktoplist = []
Ktop = len(wt)//3#arbitrary integer for now
C = 0.1
accuracy = 0
n = int(len(wt) * C)
def mifl_aggregation(wt, MI, pairs):
  ###initialize w0
  # Run on FL server
  # wt is a list of local model weights from clients
  # MI is a list of mutual information values from clients
  t = 0 #round
  global accuracy
  while accuracy < t_acc:
    selected_elements = random.choices(wt, k=n)
    for k in range(len(selected_elements)):
      wt[k],MI[k] = mifl_update(k,t,wt[k])
      t = t + 1
    for k in range(len(wt)):
      pairs[k] = [MI[k],wt[k],k]
    sorted_pairs = sorted(pairs, key=lambda x: x[0])
    # when running the algorithm for the mnist dataset, we have divided the data such that there is an
    # equal number of samples for each client, update code if that is not the case.
    for k in range(Ktop):
      ktoplist.append(sorted_pairs[k][1])
    averaged_weights = np.average(ktoplist, axis=0)
    for k in range(Ktop,len(sorted_pairs)):
      sorted_pairs[k][1] = averaged_weights
    pairs = sorted(sorted_pairs, key = lambda x: x[2])
    global_model.set_weights(averaged_weights)
    accuracy = global_model.evaluate(x_test, y_test, verbose=0)[1]
    print(accuracy)
mifl_aggregation(wt, MI, pairs)
for i in range (num_clients):
  mifl_update(i,0,wt)
  wt.append(local_models[i].get_weights())
for k in range (len(local_models)):
  wt.append(local_models[k].get_weights())

#problems currently identified in second algo-
#initialize w0 not done, what do i initialize them with?
#Ktop arbitrarily taken, step 9 -TEMP FIX
#assumed each client has equal number of samples -TEMP FIX
#once sorted, take them back to old position -SOLVED
#how is accuracy being updated, not mentioned in the algo -SOLVED
#prevent overwriting
#problems in first algorithm
#didnt take batches into account
#have to see how to integrate the first and the second








In [ ]:
type(weights)

NameError: name 'weights' is not defined

In [ ]:
#import libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mutual_info_score

#define hyperparameters
num_clients = 10

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255

# Split data into shards for each client
data_per_client = len(x_train) // num_clients
x_train_shards = np.split(x_train[:num_clients * data_per_client], num_clients)
y_train_shards = np.split(y_train[:num_clients * data_per_client], num_clients)

#algorithm 1, mifl update
# Define some hyperparameters
E = 10 # Number of local epochs
B = 32 # Batch size
h = 0.01 # Learning rate
s = 0.5 # Threshold for cv
lamb = 0#lambda
omega = 0
def create_model():
  model = keras.Sequential(
      [
          keras.Input(shape=(28, 28, 1)),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(10, activation="softmax"),
      ]
  )
  return model

# Initialize global model
global_model = create_model()
global_model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(h), metrics=["accuracy"])

'''def mutual_information_score(y_true, y_pred):
  # Compute the mutual information between the true labels and the predictions
  mi = mutual_info_score(y_true, y_pred)
  # Return the mutual information
  return mi'''
from scipy.stats import gaussian_kde
import numpy as np

def estimate_pdf(weights):
    # This function estimates the probability density function using Gaussian Kernel Density Estimation
    kde = gaussian_kde(weights)
    return kde

def kl_divergence(p, q):
    # This function calculates the Kullback-Leibler divergence between two distributions
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def mutual_information_score(weights1, weights2):
    # Estimate the PDFs for both sets of weights
    pdf1 = estimate_pdf(weights1)
    pdf2 = estimate_pdf(weights2)

    # Evaluate the PDFs on a grid of points (for simplicity, we assume 1D weights)
    grid_points = np.linspace(min(min(weights1), min(weights2)), max(max(weights1), max(weights2)), num=100)
    p1 = pdf1(grid_points)
    p2 = pdf2(grid_points)

    # Calculate the joint distribution as the outer product of the two PDFs
    joint_pdf = np.outer(p1, p2)

    # Calculate the product of the marginals
    product_marginals = p1[:, None] * p2[None, :]

    # Calculate the mutual information
    mi = kl_divergence(joint_pdf.ravel(), product_marginals.ravel())

    return mi


#list for storing current local models
local_models = [0]*(num_clients+1)

def cv_fn(lg, lk):
  # Use the formula from the paper to calculate cv
  lg_mean = tf.reduce_mean(lg, axis=0)
  lk_mean = tf.reduce_mean(lk, axis=0)
  lg_var = tf.math.reduce_variance(lg, axis=0)
  lk_var = tf.math.reduce_variance(lk, axis=0)
  lg_lk_cov = tf.reduce_mean((lg - lg_mean) * (lk - lk_mean), axis=0)
  rho = (lg_var + lk_var - 2 * lg_lk_cov) / ((lg_var * lk_var)**2)
  cv = -0.5 * tf.math.log(1-rho**2)
  return cv

def mifl_update(k,t,wt):
  if(t == 0):
    # Create a local model for the client
    Fg = create_model()
    Fg.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    # Copy the global model weights to the local model
    #Fg.set_weights(global_model.get_weights())
    Fg.fit(x_train_shards[k], y_train_shards[k], batch_size=B, epochs=E, verbose=0)
    # Add the local model to the dict
    local_models[k] = Fg
    weights = Fg.get_weights()
    mi = 0
  else:
    Fk = local_models[k]
    Fg = create_model()
    Fg.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")
    weights = global_model.get_weights()
    Fg.set_weights(weights)
    #currently not taking batches into account
    for i in range (E):
      '''range is 6000 because of number of samples in each x_train_shard,y_train_shard, change it accordingly when batches have been taken into account
      for j in range(0,6000):
          lg = Fg.predict(x_train_shards[j]) - y_train_shards[j]
          lk = Fk.predict(y_train_shards[j]) - y_train_shards[j]
      '''
      lg = Fg.predict(x_train_shards) - y_train_shards
      lk = Fk.predict(x_train_shards) - y_train_shards
      cv = cv_fn(lg,lk)
      #cv = mutual_information_score(lg,lk)
      if lg == lk:
        lamb = 1/2
      elif cv <= 1/2:
        lamb = cv
      else:
        lamb = 1 - cv
      omega = (1-lamb)*lg + lamb*lk
      weights = weights - h*omega
    #prediction = model.predict(input_sample)
    Fg.set_weights(weights)
    mi = mutual_information_score(Fg.get_weights(),Fk.get_weights())
    print(type(mi))
    print(type(weights))
  return weights,mi
  import random
# Define the MIFL-Aggregation function
t_acc = 95#targeted accuracy
wt = [0]*num_clients
MI = []
pairs = []
ktoplist = []
Ktop = len(wt)//3#arbitrary integer for now
C = 0.1
accuracy = 0
n = int(len(wt) * C)
def mifl_aggregation(wt, MI, pairs):
  ###initialize w0
  # Run on FL server
  # wt is a list of local model weights from clients
  # MI is a list of mutual information values from clients
  t = 0 #round
  global accuracy
  while accuracy < t_acc:
    selected_elements = random.choices(wt, k=n)
    for k in range(num_clients):
      wt[k],MI[k] = mifl_update(k,t,wt[k])
    t = t + 1
    for k in range(len(wt)):
      pairs[k] = [MI[k],wt[k],k]
    sorted_pairs = sorted(pairs, key=lambda x: x[0])
    # when running the algorithm for the mnist dataset, we have divided the data such that there is an
    # equal number of samples for each client, update code if that is not the case.
    for k in range(Ktop):
      ktoplist.append(sorted_pairs[k][1])
    averaged_weights = np.average(ktoplist, axis=0)
    for k in range(Ktop,len(sorted_pairs)):
      sorted_pairs[k][1] = averaged_weights
    pairs = sorted(sorted_pairs, key = lambda x: x[2])
    global_model.set_weights(averaged_weights)
    accuracy = global_model.evaluate(x_test, y_test, verbose=0)[1]
    print(accuracy)
mifl_aggregation(wt, MI, pairs)
for i in range (num_clients):
  mifl_update(i,0,wt)
  print("first round done")
  wt[i] = local_models[i].get_weights()
for k in range (len(local_models)):
  wt[i] = local_models[k].get_weights()

#problems currently identified in second algo-
#initialize w0 not done, what do i initialize them with?
#Ktop arbitrarily taken, step 9 -TEMP FIX
#assumed each client has equal number of samples -TEMP FIX
#once sorted, take them back to old position -SOLVED
#how is accuracy being updated, not mentioned in the algo -SOLVED
#prevent overwriting
#problems in first algorithm
#didnt take batches into account
#have to see how to integrate the first and the second






